<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 35kB/s 
     |████████████████████████████████| 3.8MB 53.3MB/s 
     |████████████████████████████████| 450kB 54.6MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.1.0rc0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [2]:
pip install keras==2.3.1

     |████████████████████████████████| 378kB 2.6MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [3]:
!kaggle competitions download -c ieee-fraud-detection

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 75.5MB/s]
 77% 45.0M/58.3M [00:01<00:00, 21.7MB/s]
100% 58.3M/58.3M [00:01<00:00, 50.3MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 105MB/s]
 63% 33.0M/52.2M [00:00<00:00, 107MB/s]
100% 52.2M/52.2M [00:00<00:00, 118MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 107MB/s]


In [5]:
pip install category_encoders

     |████████████████████████████████| 102kB 2.3MB/s 


In [6]:
import pandas as pd
import numpy as np
trn_trans=pd.read_csv('train_identity.csv.zip')
trn=pd.read_csv('train_transaction.csv.zip')
tst_trans=pd.read_csv('test_identity.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')
fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')
dk={}
for i in range(1,39):
    if i<10:
        dk['id-0'+str(i)]='id_0'+str(i)
    else:
        dk['id-'+str(i)]='id_'+str(i)
fnl_tst=fnl_tst.rename(columns=dk)
import gc
del([trn_trans,trn,tst_trans,tst])
gc.collect()
fnl=pd.concat([fnl_trn,fnl_tst],0)
del([fnl_trn,fnl_tst])
gc.collect()
ids=list(fnl.filter(regex='id_'))
fnl[ids]=fnl[ids].fillna(-1)
fnl[['addr1','addr2']]=fnl[['addr1','addr2']].fillna(-1)
ids=list(fnl.filter(regex='card'))
fnl[ids]=fnl[ids].fillna(-1)
ids=list(fnl.filter(regex='V'))
fnl[ids]=fnl[ids].fillna(-1)
fnl[['addr1','addr2','DeviceInfo','DeviceType','P_emaildomain','R_emaildomain']]=fnl[['addr1','addr2','DeviceInfo','DeviceType','P_emaildomain','R_emaildomain']].fillna('nan')
ids=list(fnl.filter(regex='D'))
fnl[ids]=fnl[ids].fillna(-1)
ids=list(fnl.filter(regex='C'))
fnl[ids]=fnl[ids].fillna(0)
ids=list(fnl.filter(regex='M'))
fnl[ids]=fnl[ids].fillna(-1)
fnl[['dist1','dist2']]=fnl[['dist1','dist2']].fillna(-1)
import re
def number(data):
  try:
      return  ' '.join(re.findall(r'\d+',data)) 
  except:
    return -1
def charac(data):
    try:
      return ''.join(re.findall("[a-zA-Z]+", data))
    except:
        return -1
fnl['DeviceInfo_char']=list(map(charac,fnl['DeviceInfo'].fillna('nan')))
fnl['DeviceInfo_number']=list(map(number,fnl['DeviceInfo'].fillna('nan')))


fnl['DeviceType_char']=list(map(charac,fnl['DeviceType'].fillna('nan')))
fnl['DeviceType_number']=list(map(number,fnl['DeviceType'].fillna('nan')))


fnl['os_char']=list(map(charac,fnl['id_30'].fillna('nan')))
fnl['os_number']=list(map(number,fnl['id_30'].fillna('nan')))


fnl['browse_char']=list(map(charac,fnl['id_31'].fillna('nan')))
fnl['browse_number']=list(map(number,fnl['id_31'].fillna('nan')))


fnl['P_1']=fnl['P_emaildomain'].str.split('.').str[0]
fnl['P_2']=fnl['P_emaildomain'].str.split('.').str[1]


fnl['R_1']=fnl['R_emaildomain'].str.split('.').str[0]
fnl['R_2']=fnl['R_emaildomain'].str.split('.').str[1]
z=fnl.loc[~(fnl['isFraud'].isna())]
from tqdm import tqdm
from category_encoders.cat_boost import CatBoostEncoder
cols=list(fnl.select_dtypes(include=object))
for col in tqdm(cols):
    cbe=CatBoostEncoder(sigma=0.01)
    cbe.fit(z[col],z['isFraud'])
    fnl[col]=cbe.transform(fnl[col],fnl['isFraud'])
del(z)
gc.collect()
fnl.select_dtypes(include=object)
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
fnl=fnl.drop(['isFraud'],1)
fnl=reduce_mem_usage(fnl)
from sklearn.preprocessing import StandardScaler,MinMaxScaler
ss=StandardScaler()
fnl=pd.DataFrame(ss.fit_transform(fnl))
mms=MinMaxScaler()
fnl=pd.DataFrame(ss.fit_transform(fnl))
fnl=reduce_mem_usage(fnl)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

100%|██████████| 43/43 [01:01<00:00,  1.42s/it]


Memory usage of dataframe is 3798.67 MB
Memory usage after optimization is: 1098.96 MB
Decreased by 71.1%
Memory usage of dataframe is 3750.30 MB
Memory usage after optimization is: 937.58 MB
Decreased by 75.0%


In [7]:

#input fed to model is original data where 10% columns have their values shuffled within themselves 

import gc
import os
gc.collect()
import numpy as np
import keras 
from math import ceil
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import Sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Dropout,BatchNormalization
from keras.layers import Add
import keras.backend as K
class DAESequence(Sequence):
    def __init__(self,df,batch_size):
        self.batch_size=batch_size
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df.values
        self.idx=[]
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        output_x=self.data[idx:last]
        length=last-idx
        rnd=np.random.randint(0,self.len_data-size,size)
        rnd[rnd>idx]+=size
        cols=np.random.randint(0,self.columns,int(self.columns*0.1))
        noise_x=output_x.copy()
        noise_x[:,cols]=self.data[rnd[:,None],cols]
        return noise_x,output_x
from keras.models import Sequential
from keras.callbacks.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.layers import BatchNormalization
#fnl=pd.concat([fnl_trn,fnl_tst],0).select_dtypes(exclude='uint8')
batch_size=1024
checkpoint_path = "training_1.hdf5" 
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback =ModelCheckpoint(checkpoint_path, 
monitor='loss',save_best_only=True,save_weights_only=True,verbose=1)
len_input_columns=fnl.shape[1]
model_dae = Sequential()
model_dae.add(Dense(units=len_input_columns,input_shape=(len_input_columns,), activation='relu', dtype='float32', name='Input'))
model_dae.add(Dense(units=int(300), activation='relu', dtype='float32',name='Hidden_1'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(200), activation='relu', dtype='float32',name='Hidden_2'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_3'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(25), activation='relu', dtype='float32',name='Hidden_4'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(2), activation='relu', dtype='float32',name='Hidden_5'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(25), activation='relu', dtype='float32',name='Hidden_6'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_7'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(200), activation='relu', dtype='float32',name='Hidden_8'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(300), activation='relu', dtype='float32',name='Hidden_9'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=len_input_columns, activation='linear', dtype='float32', name='Output'))
model_opt = keras.optimizers.SGD(lr=0.001, decay=1-0.9, momentum=0.7, nesterov=False)
model_dae.compile(loss='mean_squared_error',optimizer=model_opt)
hist=model_dae.fit_generator(DAESequence(fnl, batch_size=2048),steps_per_epoch=int(ceil(fnl.shape[0]/batch_size)),epochs=1000,workers=2)
model_dae.save_weights('autoenc.hdf5')

Using TensorFlow backend.


Epoch 1/1000
1072/1072 [==============================] - 124s 116ms/step - loss: 1.6939
Epoch 2/1000
1072/1072 [==============================] - 122s 114ms/step - loss: 1.6638
Epoch 3/1000
1072/1072 [==============================] - 121s 113ms/step - loss: 1.6533
Epoch 4/1000
 389/1072 [=========>....................] - ETA: 1:17 - loss: 1.6692

KeyboardInterrupt: ignored